# Stage 2: LR Sweep (Baseline vs Elementwise)

Runs short LR sweep with warmup/early-stop. Baseline (none) vs elementwise gating.

## 0a) (Once per machine) Register venv kernel
If Jupyter isn't using the venv, run this once, then switch kernel to `ese3060 venv`.

In [ ]:
%%bash
source /workspace/ese-3060-project/.venv/bin/activate
python -m ipykernel install --user --name ese3060-venv --display-name "ese3060 venv"
echo "Kernel installed."



## 0) Setup
Assumes repo at `/workspace/ese-3060-project` and data symlinked at `data/fineweb10B`. Activate venv before starting JupyterLab.

In [1]:
import os, subprocess, json, glob, re
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%cd /workspace/ese-3060-project
RESULTS = os.path.join('experiments','results.csv')


/workspace/ese-3060-project


## 1) Launch runs (Stage 2 LR sweep)
Baseline (attn_gate=none) vs elementwise, SDPA position, sigmoid gate. Warmup 150, total iters default 800, warmdown set to fit short run.

In [4]:

# Runtime knobs
NPROC = None                # set to int to override; otherwise auto-detect GPU count
BASE_LR = 0.0036
LR_MULTS = [1.0, 1.1, 1.2, 1.3]
SEEDS = [1337, 2337]
NUM_ITER = 800
WARMUP_ITERS = 150
WARMDOWN_ITERS = 600       # set <= NUM_ITER
EARLY_STOP_PATIENCE = 2    # 0 disables
EARLY_STOP_MIN_DELTA = 0.0
ATTN_GATES = ["none", "elementwise"]
SCRIPT_PATH = "/workspace/ese-3060-project/train_gpt.py"
TORCHRUN = "torchrun"
LAUNCH = True             # set True to launch runs

# torchrun helpers
if NPROC is None:
    try:
        gpu_count = int(subprocess.check_output("nvidia-smi --list-gpus | wc -l", shell=True).decode().strip())
    except Exception:
        gpu_count = 0
    NPROC = max(gpu_count, 1)

assert os.path.exists(SCRIPT_PATH), f"Missing train script: {SCRIPT_PATH}"

import shlex, signal

def run_job(attn_gate, lr, seed):
    env = os.environ.copy()
    env.update({
        "ATTNGATE": attn_gate,
        "GATEPOS": "sdpa",
        "GATEACT": "sigmoid",
        "LR": str(lr),
        "SEED": str(seed),
        "WARMUP_ITERS": str(WARMUP_ITERS),
        "NUM_ITERATIONS": str(NUM_ITER),
        "WARMDOWN_ITERS": str(WARMDOWN_ITERS),
        "EARLY_STOP_PATIENCE": str(EARLY_STOP_PATIENCE),
        "EARLY_STOP_MIN_DELTA": str(EARLY_STOP_MIN_DELTA),
    })
    cmd = [TORCHRUN, "--standalone", f"--nproc_per_node={NPROC}", SCRIPT_PATH]
    print(f"\n>>> Launching attn_gate={attn_gate} lr={lr:.5f} seed={seed} nproc={NPROC}")
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, env=env, bufsize=1, preexec_fn=os.setsid)
    for line in proc.stdout:
        print(line.rstrip())
    ret = proc.wait()
    print(f"<exit {ret}> attn_gate={attn_gate} lr={lr:.5f} seed={seed}")
    return ret

if LAUNCH:
    for attn in ATTN_GATES:
        for mult in LR_MULTS:
            lr = BASE_LR * mult
            for seed in SEEDS:
                run_job(attn, lr, seed)
else:
    print("Launch skipped (set LAUNCH=True to run)")




>>> Launching attn_gate=none lr=0.00360 seed=1337 nproc=8
W1207 16:34:24.785000 498 torch/distributed/run.py:774]
W1207 16:34:24.785000 498 torch/distributed/run.py:774] *****************************************
W1207 16:34:24.785000 498 torch/distributed/run.py:774] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
W1207 16:34:24.785000 498 torch/distributed/run.py:774] *****************************************
using device: cuda:6
using device: cuda:1
using device: cuda:4
using device: cuda:3
using device: cuda:0
using device: cuda:7
using device: cuda:2
using device: cuda:5
Training DataLoader: total number of tokens: 900000000 across 9 files
Validation DataLoader: total number of tokens: 100000000 across 1 files
step:0/800 val_loss:16.0297 train_time:251ms step_avg:nanms
step:1/800 train_loss:16.0220 train_time:75229ms step_

## 2) Load results

In [13]:

if os.path.exists(RESULTS):
    df = pd.read_csv(RESULTS)
else:
    df = pd.DataFrame()
df.head() if not df.empty else df



,run_id,date,git_commit,seed,attn_gate,gate_pos,gate_act,learning_rate,batch_size,device_batch_size,...,num_iterations,warmdown_iters,final_val_loss,best_val_loss,train_time_ms,ms_per_step,gpu_name,n_gpus,runpod_instance,notes
0,f1a4d6bb-348b-4783-8bbf-52e0c71e71e0,2025-12-07 09:42:24,21aae13b20675947154a15b640706eb3a47e5fcd,1337,none,sdpa,sigmoid,0.0036,512,64,...,1500,1450,3.533581,3.533581,593163.228989,397.829127,NVIDIA A100 80GB PCIe,8,unknown,NaN
1,e3a7db34-8900-408e-9be9-7651dab150ce,2025-12-07 09:55:12,21aae13b20675947154a15b640706eb3a47e5fcd,1338,none,sdpa,sigmoid,0.0036,512,64,...,1500,1450,3.533149,3.533149,592636.538506,397.475881,NVIDIA A100 80GB PCIe,8,unknown,NaN
2,02f2de8a-3f1e-4941-97b6-fdd9a61c21b1,2025-12-07 10:08:50,21aae13b20675947154a15b640706eb3a47e5fcd,1337,headwise,sdpa,sigmoid,0.0036,512,64,...,1500,1450,3.531132,3.531132,616917.419434,413.760845,NVIDIA A100 80GB PCIe,8,unknown,NaN
3,ee4336d9-19cb-4f01-acb6-7408e5888499,2025-12-07 10:22:08,21aae13b20675947154a15b640706eb3a47e5fcd,1338,headwise,sdpa,sigmoid,0.0036,512,64,...,1500,1450,3.531181,3.531181,614547.306776,412.171232,NVIDIA A100 80GB PCIe,8,unknown,NaN
4,e4c15308-00ba-4a79-bb37-4694c599046c,2025-12-07 10:36:18,21aae13b20675947154a15b640706eb3a47e5fcd,1337,elementwise,sdpa,sigmoid,0.0036,512,64,...,1500,1450,3.516514,3.516514,626626.270294,420.272482,NVIDIA A100 80GB PCIe,8,unknown,NaN


### Filter to Stage 2 runs (short LR sweep)

In [14]:

stage2 = df.copy()
stage2 = stage2[(stage2.get('num_iterations',0)==NUM_ITER) & (stage2.get('warmup_iters',0)==WARMUP_ITERS)]
if 'warmdown_iters' in stage2.columns:
    stage2 = stage2[stage2['warmdown_iters']==WARMDOWN_ITERS]
stage2.head()



,run_id,date,git_commit,seed,attn_gate,gate_pos,gate_act,learning_rate,batch_size,device_batch_size,...,num_iterations,warmdown_iters,final_val_loss,best_val_loss,train_time_ms,ms_per_step,gpu_name,n_gpus,runpod_instance,notes


### Aggregate metrics by config

In [15]:

if not stage2.empty:
    group_cols = ['attn_gate','learning_rate']
    agg = stage2.groupby(group_cols).agg(
        runs=('run_id','count'),
        mean_best_val=('best_val_loss','mean'),
        std_best_val=('best_val_loss','std'),
        mean_ms_step=('ms_per_step','mean'),
    ).reset_index()
    display(agg)
    if not agg.empty:
        fig, ax = plt.subplots(figsize=(6,4))
        for gate, sub in agg.groupby('attn_gate'):
            ax.plot(sub['learning_rate'], sub['mean_best_val'], marker='o', label=gate)
        ax.set_xlabel('learning_rate')
        ax.set_ylabel('best_val_loss')
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.show()
else:
    print('No stage2 rows found; run jobs first.')



No stage2 rows found; run jobs first.


### Parse val_loss curves from logs (first run per config)

In [9]:

VAL_RE = re.compile(r"step:(\d+)/(\d+).*val_loss:([0-9.]+).*train_time:(\d+)ms")
rows = []
for path in glob.glob('logs/*.txt'):
    run_id = os.path.splitext(os.path.basename(path))[0]
    # map run_id to config
    cfg = stage2[stage2['run_id']==run_id]
    if cfg.empty:
        continue
    cfg = cfg.iloc[0]
    with open(path) as f:
        for line in f:
            m = VAL_RE.search(line)
            if m:
                step = int(m.group(1))
                loss = float(m.group(3))
                t_ms = int(m.group(4))
                rows.append({'run_id': run_id, 'step': step, 'val_loss': loss, 'train_time_ms': t_ms, 'attn_gate': cfg['attn_gate'], 'learning_rate': cfg['learning_rate']})
curves = pd.DataFrame(rows)
if not curves.empty:
    fig, axes = plt.subplots(1, len(curves['attn_gate'].unique()), figsize=(12,4), sharey=True)
    if len(curves['attn_gate'].unique())==1:
        axes=[axes]
    for ax, (gate, sub) in zip(axes, curves.groupby('attn_gate')):
        for lr, sublr in sub.groupby('learning_rate'):
            ax.plot(sublr['train_time_ms']/1000.0, sublr['val_loss'], label=f"lr={lr:.5f}")
        ax.set_title(f"attn_gate={gate}")
        ax.set_xlabel('train_time (s)')
        ax.set_ylabel('val_loss')
        ax.legend()
        ax.grid(True, alpha=0.3)
    plt.show()
else:
    print('No matching logs parsed; run jobs first.')



No matching logs parsed; run jobs first.
